<a href="https://colab.research.google.com/github/mohit-saini-2001/Sarcasm_Detection/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
#importing libraries
import zipfile
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

## **Data Loading & Preprocessing**

In [5]:
#Place kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
#Downloading dataset from kaggle
!pip install kaggle
!kaggle datasets download -d danofer/sarcasm

Dataset URL: https://www.kaggle.com/datasets/danofer/sarcasm
License(s): copyright-authors
 89% 192M/216M [00:00<00:00, 504MB/s] 
100% 216M/216M [00:00<00:00, 469MB/s]


In [7]:
# Unzipping folder
sarc = zipfile.ZipFile('/content/sarcasm.zip')
sarc.extractall('/content')
sarc.close()

In [8]:
df = pd.read_csv('/content/train-balanced-sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [9]:
df = df.drop(['author', 'subreddit', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'], axis=1)
df.head()

,label,comment,score
0,0,NC and NH.,2
1,0,You do know west teams play against west teams...,-4
2,0,"They were underdogs earlier today, but since G...",3
3,0,"This meme isn't funny none of the ""new york ni...",-8
4,0,I could use one of those tools.,6


In [10]:
df = df[:10000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    10000 non-null  int64 
 1   comment  9999 non-null   object
 2   score    10000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 234.5+ KB


In [11]:
df.dropna(inplace = True)
df.isna().sum()

,0
label,0
comment,0
score,0


***See what significance has the column score with label***

In [14]:
# Finding out correlation of "score" with label column
corr_matrix = df[['label', 'score']].corr()
print(corr_matrix)

          label     score
label  1.000000 -0.019025
score -0.019025  1.000000


In [15]:
df = df.drop(['score'], axis=1)

In [16]:
# Remove symbols and number from comment column
df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [17]:
#LowerCase all comment
df['comment'] = df['comment'].str.lower()
df.head()

,label,comment
0,0,nc and nh
1,0,you do know west teams play against west teams...
2,0,they were underdogs earlier today but since gr...
3,0,this meme isnt funny none of the new york nigg...
4,0,i could use one of those tools


# **Tokenization**

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
#function for tokenization
def tokenize_data(text, max_length = 100):
  return tokenizer(
      text.tolist(),
      max_length = max_length,
      truncation = True,
      padding = 'max_length',
      return_tensors = 'np'
  )

tokenized_data = tokenize_data(df['comment'])

**Train Test Split**

In [25]:
X = tokenized_data['input_ids']
y = df['label']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((7999, 100), (2000, 100))

# **Model Building**

In [29]:
class HierarchicalBERT(tf.keras.Model):
    def __init__(self,bert_model,lstm_units,cnn_filters,dense_units):
        super(HierarchicalBERT,self).__init__()
        self.bert = bert_model
        # Sentence Encoding Layer
        self.dense_sentence = tf.keras.layers.Dense(768, activation='relu')
        # Context Summarization Layer
        self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()
        #Context Encoder Layer
        self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        # CNN Layer
        self.conv = tf.keras.layers.Conv1D(cnn_filters, 2, activation='relu')
        self.pool = tf.keras.layers.GlobalMaxPooling1D()
        # Fully Connected Layer
        self.dense_fc = tf.keras.layers.Dense(dense_units, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # BERT Embeddings
        bert_output = self.bert(inputs)[0]

        # Sentence Encoding Layer
        sentence_encoded = self.dense_sentence(bert_output)

        # Context Summarization Layer
        context_summarized = self.mean_pooling(sentence_encoded)

        # Expand dimensions to match the input shape required by LSTM
        context_summarized = tf.expand_dims(context_summarized, 1)

        # Context Encoder Layer
        context_encoded = self.bilstm_encoder(context_summarized)

        # Squeeze the second dimension to match the input shape required by Conv1D
        context_encoded_squeezed = tf.squeeze(context_encoded, axis=1)

        # Add the channels dimension to match the input shape required by Conv1D
        context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis=-1)

        # CNN Layer
        conv_output = self.conv(context_encoded_expanded)
        pooled_output = self.pool(conv_output)

        # Fully Connected Layer
        dense_output = self.dense_fc(pooled_output)

        # Output Layer
        final_output = self.output_layer(dense_output)
        return final_output

In [33]:
# pip install -U transformers tensorflow huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 582.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [39]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [40]:
#defining the hierarchical bert model
model = HierarchicalBERT(bert_model, lstm_units = 128, cnn_filters = 64, dense_units = 32)

In [41]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

In [42]:
model.fit(X_train, y_train, epochs = 3, batch_size = 32)

Epoch 1/3


 13/250 [>.............................] - ETA: 3:12:10 - loss: 0.6717 - accuracy: 0.6154

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy : {accuracy * 100}')